In [12]:
import websocket
import json
import gym
from gym import spaces
import numpy as np
import numexpr as ne

In [13]:
websocket.enableTrace(True)
ws=websocket.WebSocket()
ws.connect("ws://10.42.0.231:80/")

--- request header ---
GET / HTTP/1.1
Upgrade: websocket
Host: 10.42.0.231
Origin: http://10.42.0.231
Sec-WebSocket-Key: S7Opfo7QrO2vKJ8ueVMADA==
Sec-WebSocket-Version: 13
Connection: Upgrade


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Server: arduino-WebSocketsServer
Upgrade: websocket
Connection: Upgrade
Sec-WebSocket-Version: 13
Sec-WebSocket-Accept: MMoTmSOWyrsj+xOwsMVnwIWn3oU=
-----------------------


In [56]:
range(0, 100)

range(0, 100)

In [60]:
import numpy as np
np.array([[1,2,3],[1,2,3]]).shape

(2, 3)

In [2]:
import websocket
import json
import gym
from gym import spaces
import numpy as np
import re
import numexpr as ne
import time
from collections import OrderedDict
import logging
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly
import json
import os
plotly.io.json.config.default_engine = 'orjson'

#steps are not implemented for now
out_dict={'leddelay':{'names':['leddelay'], 'value_range':{'min':1, 'max':10001, 'step':100}, 'init_val':{'leddelay':10}},
          'ledcontrols':{'names':['lv1r','lv1g','lv1b','lv2r','lv2g','lv2b','lv3r','lv3g','lv3b','lv4r','lv4g','lv4b', 'lv5r','lv5g','lv5b','lv6r','lv6g',
          'lv6b',
          'lv7r',
          'lv7g',
          'lv7b',
          'lv8r',
          'lv8g',
          'lv8b'], 'value_range':{'min':10, 'max':255, 'step':10}, 'init_val':{'lv1r':255, 'lv1g':0, 'lv1b':0, 
                                                                               'lv2r':0, 'lv2b':255, 'lv2b':0, 
                                                                               'lv3r':0, 'lv3g':0, 'lv3b':255, 
                                                                               'lv4r':255, 'lv4g':255, 'lv4b':255, 
                                                                               'lv5r':0,   'lv5g':255, 'lv5b':255, 
                                                                               'lv6r':255, 'lv6g':0,   'lv6b':255, 
                                                                               'lv7r':255, 'lv7b':255, 'lv7b':0, 
                                                                               'lv8r':255, 'lv8g':0, 'lv8b':0}},
          'sound_wave_frequencies':{'names':['wave_1_freq','wave_2_freq'], 'value_range':{'min':1, 'max':30000, 'step':100}, 'init_val':{'wave_1_freq':440, 
                                                                                                                                         'wave_2_freq':440}},
          'panner_phasor_frequencies':{'names':['panner_freq', 'phasor_1_freq', 'phasor_2_freq','phasor_1_min',  'phasor_2_min', 'phasor_1_dif', 'phasor_2_dif'],  'value_range':{'min':1, 'max':50, 'step':1},
                                       'init_val':{'panner_freq':1,
                                                    'phasor_1_freq':10,
                                                    'phasor_2_freq':10,
                                                    'phasor_1_min':1,
                                                    'phasor_2_min':1,
                                                    'phasor_1_dif':30,
                                                    'phasor_2_dif':30}},
          'panner_div':{'names':['panner_div'], 'value_range':{'min':1, 'max':5, 'step':1}, 'init_val':{'panner_div':2}},
          'sound_wave_shapes':{'names':['wave_1_type', 'wave_2_type'], 'value_range':{'min':0, 'max':3, 'step':1}, 
                               'init_val':{'wave_1_type':0,
                                           'wave_2_type':0}},
          'maxibolume':{'names':['maxivolume'], 'value_range':{'min':0, 'max':50, 'step':10}, 
                        'init_val':{'maxivolume':10}}
}
out_order=['lv1r','lv1g','lv1b','lv2r','lv2g','lv2b','lv3r','lv3g','lv3b','lv4r','lv4g','lv4b','lv5r','lv5g','lv5b','lv6r','lv6g','lv6b',
          'lv7r','lv7g','lv7b','lv8r','lv8g','lv8b','leddelay','wave_1_freq','wave_2_freq','panner_freq','panner_div','phasor_1_freq',
          'phasor_1_min','phasor_1_dif','phasor_2_freq','phasor_2_min','phasor_2_dif','maxivolume','wave_1_type','wave_2_type']


class SFSystemCommunicator(gym.Env):
    def __init__(self, out_dict=out_dict, out_order=out_order,n_input_channels=8, channels_of_interest_inds=list(range(8)), n_timepoints_per_sample=100, max_sfsystem_output=1023,reward_formula_string='(fbin_05_4_ch0+freq_30_ch0)/fbin_12_30_ch0', 
                 fbins=[(0.001, 0.5), (0.5, 4), (4,8), (8,12), (12,30)], delay=10,
                 use_raw_in_os_def=False, use_freq_in_os_def=False, use_fbins_in_os_def=False, device_address="ws://10.42.0.231:80/",
                 step_stim_length=10000, episode_time_seconds=60, log_and_plot_data=True, render_data=True, return_plotly_figs=False,
                 logfn='current_training.log', write_log=True, write_signals=True):
        self.device_address=device_address
        self.step_stim_length=step_stim_length
        self.episode_time_seconds=episode_time_seconds
        self.n_steps_per_episode=int(self.episode_time_seconds*1000/self.step_stim_length)
        self.cur_step=0
        self.render_data=render_data
        self.return_plotly_figs=return_plotly_figs
        
        self.out_dict=out_dict
        self.out_order=out_order
        self.max_sfsystem_output=max_sfsystem_output
        self.n_timepoints_per_sample=n_timepoints_per_sample
        self.n_input_channels=n_input_channels
        self.reward_formula_string=reward_formula_string

        self.delay=delay

        self.record_raw=use_raw_in_os_def

        self.do_fft=use_freq_in_os_def
        self.record_fft=use_freq_in_os_def

        self.do_fbins=use_fbins_in_os_def
        self.record_fbins=use_fbins_in_os_def

        self.fbins=fbins
        self.n_fbins=len(self.fbins)

        self.channels_of_interest_inds=channels_of_interest_inds
        self.channels_of_interest_inds.sort()
        self.n_channels_of_interest=len(self.channels_of_interest_inds)
    

        self.set_fft_params()

        self.timesleep_period=0.1

        if 'raw' in self.reward_formula_string:
            self.record_raw=True
        if 'freq' in self.reward_formula_string:
            self.record_fft=True
        if 'fbin' in self.reward_formula_string:
            self.record_fbins=True

        if self.record_fbins:
            self.do_fft=True
            self.do_fbins=True
        
        self.init_action_space()
        self.init_observation_space()
        self.set_value_dict_for_reward_function()
        self.connect()
        print(self.connection_status)
        self.set_delay_and_data_transfer_buffer_size()
        print('Delay and data transfer buffer size are set up.')
        self.set_default_actions()
        print('Default actions are set.')


        #logging part
        self.logfn=logfn
        self.write_log=write_log
        self.write_signals=write_signals
        self.cur_episode_rewards=[]
        self.previous_episodes_max_rewards=[]
        self.log_and_plot_data=log_and_plot_data
        self.episode_max_reward=0
        self.current_episode=0
        self.set_fbin_x_axis_labels()
        if self.log_and_plot_data:
            if self.write_log:
                logging.basicConfig(filename=self.logfn, level=logging.INFO)

    def help(self):
        print('Reward formula can use the following operators: //, *, **, -, +')
        print('It can refer to channels using "ch" prefix followed by an index (starting with 0) e.g. ch0')
        print('It can refer to values of frequency bins in specific channels e.g. fbin_10_50_ch0')
        print('The corresponding freqency bins must be present among fbins passed at the initialization step')
        print('fbins should be passed in the form of [(b1 min, b1max),...(bn min, bn max)]')
        print('It can refer to specific frequencies from fft e.g. freq_50_ch0')
        print('For fractional frequencies (e.g. 0.5) do not use points, just start with 0 (e.g. 05)')
        print('Some examples:')
        print('1. (freq_50_ch0+fbin_0_10_ch0)/(fbin_20_30_ch0)')
        print('2. freq_5_ch0/freq_10_ch0')
        print('3. fbin_05_5_ch0')
    def clear_log(self):
        if os.path.isfile(self.logfn):
            os.remove(self.logfn)
    def set_value_dict_for_reward_function(self):
        ftokens=re.split(r'[+/)(*]+',self.reward_formula_string)
        self.rewarddict={}
        self.tokendict={}
        for token in ftokens:
            if 'ch' in token:
                self.rewarddict[token]=None
                self.tokendict[token]={}
                subtokens=token.split('_')
                self.tokendict[token]['datatype']=subtokens[0]
                for subtoken in subtokens:
                    if 'ch' in subtoken:
                        self.tokendict[token]['channelindex']=int(subtoken.split('h')[1])
                    if subtoken=='freq':
                        tfreq=float(subtokens[1])
                        self.tokendict[token]['freqdata']=tfreq
                        closestind=np.argmin(np.abs(self.f_plot - tfreq))
                        self.tokendict[token]['closest_fft_ind']=closestind
                        print(f'Token {token}:')
                        print(f'Closest fft frequency {self.f_plot[closestind]}')
                    if subtoken=='fbin':
                        bin_lst=[subtokens[1],subtokens[2]]
                        for i in range(2):
                            val=bin_lst[i]
                            if val.startswith('0'):
                                val=float('0.'+val[1:])
                            else:
                                val=float(val)
                            bin_lst[i]=val
                        self.tokendict[token]['freqdata']=tuple(bin_lst)
                        self.tokendict[token]['fbin_idx']=self.fbins.index(self.tokendict[token]['freqdata'])
    def set_default_actions(self):
        observation_space_sample=self.observation_space.sample()
        for key1 in observation_space_sample:
            for key2 in self.out_dict:
                if key1 in self.out_dict[key2]['init_val']:
                    observation_space_sample[key2]=self.out_dict[key1]['init_val'][key2]

        self.default_actions=observation_space_sample
    def populate_rewarddict(self, observations):
        for token, tokendata in self.tokendict.items():
            tartype=tokendata['datatype']
            tarchannelidx=tokendata['channelindex']
            if tartype=='raw':
                tarobs=observations['raw_data']
                tarobs=tarobs[:,tarchannelidx]
                res=np.mean(tarobs) #here we use the mean, but this may be changed
            if tartype=='freq':
                #tarfreq=tokendata['freqdata']
                taridx=tokendata['closest_fft_ind']
                tarobs=observations['fft'][tarchannelidx]
                res=tarobs[taridx]
            if tartype=='fbin':
                tarobs=observations['fbins'][tarchannelidx]
                res=tarobs[tokendata['fbin_idx']]
            self.rewarddict[token]=res
    def get_reward(self, observations=None, toreturn=False):
        if observations == None:
            observations=self.observation_space.sample() #if no observations are given extrernally, sample from the observation space
        self.populate_rewarddict(observations)
        self.reward=ne.evaluate(self.reward_formula_string, local_dict=self.rewarddict)
        if toreturn==True:
            return self.reward
    def init_action_space(self):
        self.action_space=spaces.Dict({})
        for key, val in self.out_dict.items():
            spacesnames=val['names']
            spacesrange=val['value_range']
            for spacename in spacesnames:
                self.action_space.spaces[spacename]=spaces.Box(low=spacesrange['min'], high=spacesrange['max'], shape=(1,), dtype=int)
    def init_observation_space(self):
        self.observation_space=spaces.Dict({})
        self.observation_space['raw_data']=spaces.Box(low=0, high=self.max_sfsystem_output, shape=(self.n_timepoints_per_sample, self.n_channels_of_interest), dtype=int) #n timepoints per sample rows, n input channels columns, signals should be normalized
        if self.record_fft:
            self.observation_space['fft']=spaces.Box(low=0.0, high=1.0, shape=(self.n_channels_of_interest, self.n_fft_values))
        if self.record_fbins:
            self.observation_space['fbins']=spaces.Box(low=0.0, high=1.0, shape=(self.n_channels_of_interest, self.n_fbins))
    def set_fft_params(self):
        self.sampling_frequency=int(1000/self.delay)
        self.max_possible_fft_frequency=self.sampling_frequency/2
        self.sampling_period=int(self.delay*self.n_timepoints_per_sample)
        self.timesteps=np.linspace(0, (self.n_timepoints_per_sample-1)*self.delay, self.n_timepoints_per_sample)
        self.fstep=self.sampling_frequency/self.n_timepoints_per_sample
        self.f=np.linspace(0, (self.n_timepoints_per_sample-1)*self.fstep, self.n_timepoints_per_sample)
        self.f_plot=self.f[0:int(self.n_timepoints_per_sample/2 + 1)]
        self.n_fft_values=len(self.f_plot)

    def connect(self):
        websocket.enableTrace(True)
        self.ws=websocket.WebSocket()
        self.ws.connect(self.device_address)
        self.connection_status=self.ws.recv()
    
    def get_fft_fromsignal(self, raw_singlech):
        X=np.fft.fft(raw_singlech)
        X_mag=np.abs(X)/self.n_timepoints_per_sample
        X_mag_plot=2*X_mag[0:int(self.n_timepoints_per_sample/2 + 1)]
        return X_mag_plot

    def get_fft_allchannels(self, raw_data=None):
        if raw_data == None:
            raw_data=self.observation_space.sample()['raw_data']

        fft_data=[]
        for chindex in range(raw_data.shape[1]):
            chraw=raw_data[:,chindex]
            chfft=self.get_fft_fromsignal(chraw)
            fft_data.append(chfft)
        fft_data=np.array(fft_data)
        return fft_data
    
    def get_bin_values_from_signal(self, fft_signlech):
        fpl=np.array(self.f_plot)
        xmp=np.array(fft_signlech)
        magnitudes=[]
        for low, high in self.fbins:
            mask = (fpl >= low) & (fpl < high)
            magnitude = np.abs(xmp[mask]).mean() #here can be other functions
            magnitudes.append(magnitude)
        magnitudes=np.array(magnitudes)
        return magnitudes
    def set_fbin_x_axis_labels(self):
        self.fbin_axis_labels=[]
        for low, high in self.fbins:
            binname=f'{low}-{high} Hz'
            self.fbin_axis_labels.append(binname)
    def get_bin_values_allchannels(self, fft=None):
        if fft == None:
            fft=self.observation_space.sample()['fft']
        fbins_data=[]
        for chindex in range(fft.shape[0]):
            chfft=fft[chindex,:]
            chbins=self.get_bin_values_from_signal(chfft)
            fbins_data.append(chbins)
        fbins_data=np.array(fbins_data)
        return fbins_data

    def set_delay_and_data_transfer_buffer_size(self):
        self.ws.send("set_delay_and_data_transfer_buffer_size")
        time.sleep(self.timesleep_period)
        setup=False
        while setup==False:
            try:
                device_msg=self.ws.recv()
                if device_msg == "Awaiting delay and data transfer buffer size in shape with space separator":
                    self.ws.send(f'{self.delay},{self.n_timepoints_per_sample}')
                time.sleep(self.timesleep_period)
                device_msg=self.ws.recv()
                if device_msg == "Delay and data transfer buffer size set up":
                    setup = True 
                    break;
            except:
                pass
    def start_data_transfer_from_device(self):
        self.ws.send("start_data_transfer_from_ads")
    def stop_data_transfer_from_device(self):
        self.ws.send("stop_data_transfer_from_ads")
    def stop_audiovis_feedback(self):
        self.ws.send("stop_led_cycle")
    def update_audiovis_feedback(self, update_dict=None):
        if update_dict == None:
            update_dict=self.action_space.sample()
        self.ws.send("receive_output_control_data")
        outmsg_vals=[]
        for controlnm in self.out_order:
            outmsg_vals.append(update_dict[controlnm][0])
        self.current_control_msg=','.join(list(map(str,outmsg_vals)))
        self.ws.send(self.current_control_msg)
    def sample_observations(self):
        self.ws.send("start_data_transfer_from_ads")
        self.current_sample=self.ws.recv()
        self.current_sample=json.loads(self.current_sample['data'])
        self.ws.send("stop_data_transfer_from_ads")
    def sample_and_process_observations_from_device(self):
        new_observations=dict()
        raw_data=self.sample_observations()
        new_observations['raw_data']=raw_data
        if self.do_fft:
         fft=self.get_fft_allchannels(raw_data=raw_data)
         if self.record_fft:
            new_observations['fft']=fft
        if self.do_fbins:
            fbins=self.get_bin_values_allchannels(fft=fft)
            if self.record_fbins:
                new_observations['fbins']=fbins 
        new_observations=OrderedDict(new_observations)
        return new_observations
    def step(self, action):
        self.update_audiovis_feedback(action)
        time.sleep(self.step_stim_length)

        new_observations=self.sample_and_process_observations_from_device()
        new_observations=new_observations[:,self.channels_of_interest_inds]
        self.cur_observations=new_observations
        reward=self.get_reward(observations=new_observations, toreturn=True)



        if self.log_and_plot_data==True:
            if reward>self.episode_max_reward:
                self.episode_max_reward=reward
            self.cur_episode_rewards.append(reward)
            if self.write_log:
                logging.info(f'Episode {self.current_episode}, Step {self.cur_step}, Reward {reward}')
                if self.log_signals==True:
                    logging.info(str(self.cur_observations))            


        if self.cur_step<self.n_steps_per_episode:
            done=False
            self.cur_step+=1
        else:
            done=True
        return new_observations, reward, done, {}
    def reset(self):
        self.stop_audiovis_feedback()
        self.cur_step=0
        new_observations=self.sample_and_process_observations_from_device()
        self.cur_observations=new_observations
        self.update_audiovis_feedback(update_dict=self.default_actions) #update back to default actions
        if self.log_and_plot_data==True:
            self.previous_episodes_max_rewards.append(self.episode_max_reward)
            self.cur_episode_rewards=[]
        time.sleep(self.step_stim_length) #prepare the brain for the next episode 
        return new_observations

    def render(self, elems=['reward_lineplots', 'current_fft', 'current_fbins']):
        figures=dict()
        if 'reward_lineplots' in elems:
            training_fig=sp.make_subplots(rows=1, cols=2)
            training_fig.add_trace(sp.go.Scatter(x=list(range(len(self.cur_episode_rewards))), y=self.cur_episode_rewards, mode='lines+markers', name='Current episode rewards'), row=1, column=1)
            training_fig.add_trace(sp.go.Scatter(x=list(range(len(self.previous_episodes_max_rewards))), y=self.previous_episodes_max_rewards, mode='lines+markers', name='Previous episode max rewards'), row=1, column=2)
            figures['reward_lineplots']=training_fig
            if self.render_data:
                training_fig.show()
        if ('current_fft' in elems) or ('current_fbins' in elems):
            if self.record_fft or self.record_fbins:
                signal_fig=sp.make_subplots(rows=self.n_channels_of_interest, cols=2)
                if 'current_fft' in elems and self.record_fft:
                    for chidx in range(self.n_channels_of_interest):
                        orig_chidx=self.channels_of_interest_inds[chidx]
                        chfft=self.cur_observations['fft'][chidx]
                        signal_fig.add_trace(sp.go.Scatter(x=self.f_plot, y=chfft, mode='lines+markers', name=f'Channel {orig_chidx} spectrum'), row=chidx+1, column=1)
                if 'current_fbins' in elems and self.record_fbins:
                    for chidx in range(self.n_channels_of_interest):
                        orig_chidx=self.channels_of_interest_inds[chidx]
                        chbins=self.cur_observations['fbins'][chidx]
                        signal_fig.add_trace(sp.go.Bar(x=self.fbin_axis_labels, y=chbins, mode='lines+markers', name=f'Channel {orig_chidx} frequency bins'), row=chidx+1, column=2)
                if self.render_data:
                    signal_fig.show()
        if self.return_plotly_figs:
            return figures
    def close(self):
        self.stop_audiovis_feedback() #just in case
        self.stop_data_transfer_from_device() #just in case
        self.cur_step=0 #just in case
        self.current_episode=0

In [3]:
communicator=SFSystemCommunicator()

--- request header ---
GET / HTTP/1.1
Upgrade: websocket
Host: 10.42.0.231
Origin: http://10.42.0.231
Sec-WebSocket-Key: sKTlFCzVOAFLDCH7PPJtjQ==
Sec-WebSocket-Version: 13
Connection: Upgrade


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Server: arduino-WebSocketsServer
Upgrade: websocket
Connection: Upgrade
Sec-WebSocket-Version: 13
Sec-WebSocket-Accept: /chj//Pvge5bCdzeYBt694JIDm4=
-----------------------


Token freq_30_ch0:
Closest fft frequency 30.0


++Rcv raw: b'\x89\x00'
++Rcv decoded: fin=1 opcode=9 data=b''
++Sent raw: b'\x8a\x80\xbf[PB'
++Sent decoded: fin=1 opcode=10 data=b''
++Rcv raw: b'\x81\tConnected'
++Rcv decoded: fin=1 opcode=1 data=b'Connected'
++Sent raw: b'\x81\xa7\xc2\xd4*h\xb1\xb1^7\xa6\xb1F\t\xbb\x8bK\x06\xa6\x8bN\t\xb6\xb5u\x1c\xb0\xb5D\x1b\xa4\xb1X7\xa0\xa1L\x0e\xa7\xa6u\x1b\xab\xaeO'
++Sent decoded: fin=1 opcode=1 data=b'set_delay_and_data_transfer_buffer_size'
++Rcv raw: b'\x81JAwaiting delay and data transfer buffer size in shape with space separator'
++Rcv decoded: fin=1 opcode=1 data=b'Awaiting delay and data transfer buffer size in shape with space separator'
++Sent raw: b'\x81\x86\xecYs\xf6\xddi_\xc7\xdci'
++Sent decoded: fin=1 opcode=1 data=b'10,100'


Connected


++Rcv raw: b'\x81*Delay and data transfer buffer size set up'
++Rcv decoded: fin=1 opcode=1 data=b'Delay and data transfer buffer size set up'


Delay and data transfer buffer size are set up.


In [3]:
communicator.sample_observations()

++Sent raw: b'\x81\x9c@\xa7\x0f|3\xd3n\x0e4\xf8k\x1d4\xc6P\x082\xc6a\x0f&\xc2}#&\xd5`\x11\x1f\xc6k\x0f'
++Sent decoded: fin=1 opcode=1 data=b'start_data_transfer_from_ads'
++Rcv raw: b'\x81~\x06\x81{"ch1":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch2":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch3":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch4":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [12]:
communicator.update_audiovis_feedback(communicator.action_space.sample())

++Sent raw: b'\x81\x9b\xd6_M\xe0\xa4:.\x85\xbf)(\xbf\xb9*9\x90\xa3+\x12\x83\xb919\x92\xb93\x12\x84\xb7+,'
++Sent decoded: fin=1 opcode=1 data=b'receive_output_control_data'
++Sent raw: b'\x81\xfe\x00\x82\xc9\xb1R\xdd\xf8\x82k\xf1\xf8\x81a\xf1\xf8\x83c\xf1\xfb\x85e\xf1\xfb\x85c\xf1\xf8\x88j\xf1\xfd\x81~\xeb\xf8\x9dd\xef\xe5\x80k\xee\xe5\x89b\xf1\xf8\x83b\xf1\xf8\x83j\xf1\xfb\x85a\xf1\xfb\x81`\xf1\xff\x81~\xe4\xf0\x9dk\xe8\xe5\x86e\xf1\xf8\x80d\xf1\xf8\x88c\xf1\xff\x85~\xe9\xfc\x9dd\xe5\xe5\x87e\xec\xf9\x9d`\xed\xf1\x82k\xf1\xfb\x86e\xec\xfe\x9d`\xec\xe5\x82~\xef\xf0\x9d`\xef\xe5\x85g\xf1\xfa\x85~\xe9\xf0\x9d`\xe5\xe5\x87~\xec\xe5\x81'
++Sent decoded: fin=1 opcode=1 data=b'139,103,121,247,241,198,40,61,62,193,80,120,128,243,202,60,99,95,77,116,191,64,45,68,6710,20839,27717,21,3,29,22,45,34,49,28,6,1,0'


In [13]:
communicator.stop_audiovis_feedback()

++Sent raw: b'\x81\x8e\xa9M\xc2\x1b\xda9\xadk\xf6!\xa7\x7f\xf6.\xbbx\xc5('
++Sent decoded: fin=1 opcode=1 data=b'stop_led_cycle'


In [8]:
communicator.current_sample

'{"ch1":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch2":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch3":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch4":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],"ch5":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [3]:
actspace=communicator.action_space.sample()

In [7]:
actspace

OrderedDict([('leddelay', array([2401])),
             ('lv1r', array([112])),
             ('lv1g', array([101])),
             ('lv1b', array([56])),
             ('lv2r', array([222])),
             ('lv2g', array([185])),
             ('lv2b', array([200])),
             ('lv3r', array([184])),
             ('lv3g', array([100])),
             ('lv3b', array([41])),
             ('lv4r', array([123])),
             ('lv4g', array([170])),
             ('lv4b', array([19])),
             ('lv5r', array([249])),
             ('lv5g', array([238])),
             ('lv5b', array([176])),
             ('lv6r', array([145])),
             ('lv6g', array([124])),
             ('lv6b', array([207])),
             ('lv7r', array([249])),
             ('lv7g', array([22])),
             ('lv7b', array([85])),
             ('lv8r', array([45])),
             ('lv8g', array([148])),
             ('lv8b', array([110])),
             ('wave_1_freq', array([4648])),
             ('wave_2_freq', ar

In [17]:
sample=communicator.observation_space.sample()
sample['raw_data'].shape[1]

8

In [18]:
sample['raw_data']

array([[ 633,  909,  253,  174,  711,  192,  812,  991],
       [ 355,  144,  985,   83,  882,  887,  214,   84],
       [ 614,  687,  346,  308,  653,  238,  810,  591],
       [ 883,  862,  456,   30,  968,    2,  301,  574],
       [ 616,  797,  605,  168,  826,   42,  486,  555],
       [ 254,  545,  324,  660,  796,  887,  792,  455],
       [ 949,  350,  269,  885,  498,  935,  178,  762],
       [ 376,  157,  471,  169,  565,  282,  429,  358],
       [ 942,  278,  955,  781,  112,  950,  288,  589],
       [ 126,  665,   77,  906,  229,  528,  848,   83],
       [ 275,  446,  179,  504,  738,  536,  249,  266],
       [ 798,  994,  226,  201,   13,  775,  836,  602],
       [ 670,  360,   60,  780,  707,  165,  380,  856],
       [ 243,  481,  710,   81,  894,  381,  544,  783],
       [ 462,  197,  161,  229,    1,  735,  381,  855],
       [ 808,  282,  185,  338,  630,  914,  589,  181],
       [ 640,   23,  489,  776,  799,  502,  452,  791],
       [  79,   86,  299,  779,

In [14]:
communicator.rewarddict

{'fbin_05_4_ch0': None, 'freq_30_ch0': None, 'fbin_12_30_ch0': None}

In [15]:
communicator.get_reward()

In [16]:
communicator.reward

array(4.1762214, dtype=float32)

In [51]:
communicator.rewarddict

{'fbin_0.5_5_ch0': None, 'freq_30_ch0': None, 'fbin_10_20_ch0': None}

In [12]:
s = "(freq_50+fbin_0_10)/(fbin_20_30)"

# Define the variables
variables = {
    'freq_50': 10,
    'fbin_0_10': 20,
    'fbin_20_30':40
}

# Evaluate the string as a mathematical expression
result = ne.evaluate(s, local_dict=variables)
result

array(0.75)

In [3]:
communicator.print_reward_formula_legend()

Reward formula can use the following operators: //, *, **, -, +
It can refer to channels using "ch" prefix followed by an index (starting with 0) e.g. ch0
It can refer to values of frequency bins in specific channels e.g. fbin_10-50|ch0
The corresponding freqency bins must be present among fbins passed at the initialization step
fbins should be passed in the form of [(b1 min, b1max),...(bn min, bn max)]
It can refer to specific frequencies from fft e.g. freq_50|ch0
Some examples:
1. (freq_50|ch0+fbin_0-10|ch0)/(fbin_20-30|ch0)
2. freq_5|ch0/freq_10|ch0
3. fbin_0.5-5|ch0


In [14]:
communicator.connection_status

'Connected'

In [ ]:



leddelay=100
ledvalues=[30, 0, 0, 0, 255, 0, 0, 0, 255, 255, 255, 255, 0, 255, 255, 255, 0, 255, 255, 255, 0, 255, 0, 0]
sdict={'wave_1_freq':800, 'wave_2_freq':200, 'panner_freq':30, 'panner_div':2, 'phasor_1_freq':100, 'phasor_1_min':30, 'phasor_1_max':30, 
       'phasor_2_freq':30, 'phasor_2_min':30, 'phasor_2_max':30,
       'maxivolume':5,'wave_1_type':3,'wave_2_type':0}


In [15]:
ws.recv()

++Rcv raw: b'\x89\x00'
++Rcv decoded: fin=1 opcode=9 data=b''
++Sent raw: b'\x8a\x80M\xc5\x07\x01'
++Sent decoded: fin=1 opcode=10 data=b''
++Rcv raw: b'\x81\tConnected'
++Rcv decoded: fin=1 opcode=1 data=b'Connected'


'Connected'

In [16]:
ws.send("receive_output_control_data")

++Sent raw: b'\x81\x9b\xe1\xd95\xcf\x93\xbcV\xaa\x88\xafP\x90\x8e\xacA\xbf\x94\xadj\xac\x8e\xb7A\xbd\x8e\xb5j\xab\x80\xadT'
++Sent decoded: fin=1 opcode=1 data=b'receive_output_control_data'


33

In [17]:
# 0. noise
# 1. sinewave
# 2. square
# 3. triangle


leddelay=100
ledvalues=[30, 0, 0, 0, 255, 0, 0, 0, 255, 255, 255, 255, 0, 255, 255, 255, 0, 255, 255, 255, 0, 255, 0, 0]
sdict={'wave_1_freq':800, 'wave_2_freq':200, 'panner_freq':30, 'panner_div':2, 'phasor_1_freq':100, 'phasor_1_min':30, 'phasor_1_max':30, 
       'phasor_2_freq':30, 'phasor_2_min':30, 'phasor_2_max':30,
       'maxivolume':5,'wave_1_type':3,'wave_2_type':0}

In [18]:
message=','.join(list(map(str,ledvalues)))+f',{leddelay}'+f",{sdict['wave_1_freq']},{sdict['wave_2_freq']},{sdict['panner_freq']},{sdict['panner_div']},{sdict['phasor_1_freq']},{sdict['phasor_1_min']},{sdict['phasor_1_max']},{sdict['phasor_2_freq']},{sdict['phasor_2_min']},{sdict['phasor_2_max']},{sdict['maxivolume']},{sdict['wave_1_type']},{sdict['wave_2_type']}"

In [19]:
message

'30,0,0,0,255,0,0,0,255,255,255,255,0,255,255,255,0,255,255,255,0,255,0,0,100,800,200,30,2,100,30,30,30,30,30,5,3,0'

In [20]:
len(message.split(','))

38

In [21]:
nsecs=60
nmessages=int(nsecs*(1000/leddelay))
for m in range(2):
    ws.send(message)

++Sent raw: b'\x81\xf2C\x9e\x08\xb5p\xae$\x85o\xae$\x85o\xac=\x80o\xae$\x85o\xae$\x87v\xab$\x87v\xab$\x87v\xab$\x87v\xab$\x85o\xac=\x80o\xac=\x80o\xac=\x80o\xae$\x87v\xab$\x87v\xab$\x87v\xab$\x85o\xac=\x80o\xae$\x85o\xaf8\x85o\xa68\x85o\xac8\x85o\xad8\x99q\xb29\x85s\xb2;\x85o\xad8\x99p\xae$\x86s\xb2;\x85o\xab$\x86o\xae'
++Sent decoded: fin=1 opcode=1 data=b'30,0,0,0,255,0,0,0,255,255,255,255,0,255,255,255,0,255,255,255,0,255,0,0,100,800,200,30,2,100,30,30,30,30,30,5,3,0'
++Sent raw: b'\x81\xf2\xf8\xa5~@\xcb\x95Rp\xd4\x95Rp\xd4\x97Ku\xd4\x95Rp\xd4\x95Rr\xcd\x90Rr\xcd\x90Rr\xcd\x90Rr\xcd\x90Rp\xd4\x97Ku\xd4\x97Ku\xd4\x97Ku\xd4\x95Rr\xcd\x90Rr\xcd\x90Rr\xcd\x90Rp\xd4\x97Ku\xd4\x95Rp\xd4\x94Np\xd4\x9dNp\xd4\x97Np\xd4\x96Nl\xca\x89Op\xc8\x89Mp\xd4\x96Nl\xcb\x95Rs\xc8\x89Mp\xd4\x90Rs\xd4\x95'
++Sent decoded: fin=1 opcode=1 data=b'30,0,0,0,255,0,0,0,255,255,255,255,0,255,255,255,0,255,255,255,0,255,0,0,100,800,200,30,2,100,30,30,30,30,30,5,3,0'


In [22]:
ws.send('stop_led_cycle')

++Sent raw: b'\x81\x8e\xaa\x15\xa0\x99\xd9a\xcf\xe9\xf5y\xc5\xfd\xf5v\xd9\xfa\xc6p'
++Sent decoded: fin=1 opcode=1 data=b'stop_led_cycle'


20

In [263]:
len("%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d".split(','))

25